In [1]:
import nltk
import pandas as pd
import sklearn
import re  
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
## For Stemming
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import os

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string

In [2]:
# moviereview_v2.csv was cleaned in hw 1

csv_file = 'moviereview_v2.csv'

df_movie = pd.read_csv(csv_file)
print(df_movie.shape)
df_movie = df_movie.dropna()
print(df_movie.shape)

(2000, 2)
(2000, 2)


In [3]:
df_movie.head()

,text,reviewclass
0,plot two teen couples go to a church party dr...,neg
1,the happy bastard s quick movie review ndamn ...,neg
2,it is movies like these that make a jaded mov...,neg
3,quest for camelot is warner bros first featur...,neg
4,synopsis a mentally unstable man undergoing p...,neg


In [4]:
reviews = df_movie["reviewclass"] # keep sentiment

df_movie.drop(["reviewclass"], axis=1, inplace=True) # drop sentiment for vectorization
df_movie.head()

,text
0,plot two teen couples go to a church party dr...
1,the happy bastard s quick movie review ndamn ...
2,it is movies like these that make a jaded mov...
3,quest for camelot is warner bros first featur...
4,synopsis a mentally unstable man undergoing p...


In [5]:
'''
my_list=[] # list to store results
shortword = re.compile(r'\W*\b\w{1,3}\b') # filtering out all words with len <= 3
for i in range(len(df_movie)):

    my_list.append(shortword.sub('', df_movie.iloc[i,0]))
'''

"\nmy_list=[] # list to store results\nshortword = re.compile(r'\\W*\x08\\w{1,3}\x08') # filtering out all words with len <= 3\nfor i in range(len(df_movie)):\n\n    my_list.append(shortword.sub('', df_movie.iloc[i,0]))\n"

In [6]:
my_list=[] # list to store results
shortword = re.compile(r'\W*\b\w{1,3}\b') # filtering out all words with len <= 3

my_list = [shortword.sub('', df_movie.iloc[i,0]) for i in range(len(df_movie))]

In [7]:
my_list[5]

' capsule 2176 planet mars police taking into custody accused murderer face title menace nthere fighting whole story otherwise njohn carpenter reprises many ideas from previous films especially assault precinct that film comes homage himself njohn carpenter apparently believes that action scenes which people fight something horrible same horror scenes nfor writer director horror films supposedly expert horror very mistake make nghosts mars called horror movie more just drawn fight between humans surprisingly powered alien menace addition anybody john carpenter made ghosts mars carpenter would have grounds nthis film just chock full pieces taken from assault precinct thing prince darkness fact surprising that carpenter managed many pieces previous work into this film such admittedly novel nbut that still does make really good science fiction experience nghosts mars takes place year 2176 nmars been mostly terraformed that humans walk surface without breathing gear which good film budget 

In [18]:
count_vec = CountVectorizer(input="content")
cv = count_vec.fit_transform(my_list)

# stolen from Dr. Gates
MyColumnNames=count_vec.get_feature_names()
VectorizedDF_Text=pd.DataFrame(cv.toarray(),columns=MyColumnNames)

NEW_Labels = reviews.to_frame()  

NEW_Labels.index =NEW_Labels.index-1

LabeledCLEAN_DF=VectorizedDF_Text
LabeledCLEAN_DF["LABEL"]=NEW_Labels

In [19]:
print(VectorizedDF_Text.shape) # check size of vecorization before

threshold = 1000 # threshold for drop
to_drop = [] # store indexes needed to be dropped

# id all indexes that don't meet threshold

'''
for i in range (len(VectorizedDF_Text.columns)-1):
    if sum(VectorizedDF_Text.iloc[:,i]) < threshold:
        to_drop.append(i)
'''        
to_drop =  [i for i in range (len(VectorizedDF_Text.columns)-1) if sum(VectorizedDF_Text.iloc[:,i]) < threshold]  

# drop columns that don't meet threshold
VectorizedDF_Text.drop(VectorizedDF_Text.columns[to_drop],axis=1,inplace=True)

print(VectorizedDF_Text.shape)  # check size of vecorization after

(2000, 42926)
(2000, 83)


In [20]:
VectorizedDF_Text.head()

,about,action,after,also,back,because,been,being,best,character,...,when,where,which,while,will,with,work,world,would,LABEL
0,2,0,2,1,1,2,2,0,0,2,...,1,0,3,1,0,5,0,2,1,neg
1,0,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,neg
2,0,0,0,0,0,1,2,1,1,0,...,1,0,2,0,0,2,0,0,1,neg
3,0,0,0,0,0,0,0,0,1,0,...,2,0,1,0,2,5,0,0,0,neg
4,5,0,1,1,0,1,1,1,0,1,...,1,1,1,1,0,1,0,0,0,neg


In [35]:
VectorizedDF_Text.iloc[:,0:82].apply(lambda x : x/len(VectorizedDF_Text)) # frequency of words


,about,action,after,also,back,because,been,being,best,character,...,what,when,where,which,while,will,with,work,world,would
0,0.0010,0.0000,0.0010,0.0005,0.0005,0.0010,0.0010,0.0000,0.0000,0.0010,...,0.0015,0.0005,0.0000,0.0015,0.0005,0.0000,0.0025,0.0000,0.0010,0.0005
1,0.0000,0.0005,0.0000,0.0000,0.0005,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0005,0.0005,0.0000,0.0000,0.0000,0.0000,0.0005,0.0005,0.0000,0.0000
2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0005,0.0010,0.0005,0.0005,0.0000,...,0.0000,0.0005,0.0000,0.0010,0.0000,0.0000,0.0010,0.0000,0.0000,0.0005
3,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0005,0.0000,...,0.0000,0.0010,0.0000,0.0005,0.0000,0.0010,0.0025,0.0000,0.0000,0.0000
4,0.0025,0.0000,0.0005,0.0005,0.0000,0.0005,0.0005,0.0005,0.0000,0.0005,...,0.0010,0.0005,0.0005,0.0005,0.0005,0.0000,0.0005,0.0000,0.0000,0.0000
5,0.0000,0.0005,0.0000,0.0005,0.0000,0.0000,0.0015,0.0000,0.0000,0.0000,...,0.0005,0.0000,0.0000,0.0010,0.0000,0.0000,0.0015,0.0005,0.0000,0.0005
6,0.0035,0.0000,0.0005,0.0000,0.0000,0.0000,0.0005,0.0005,0.0000,0.0005,...,0.0010,0.0005,0.0000,0.0000,0.0000,0.0000,0.0005,0.0000,0.0005,0.0000
7,0.0000,0.0000,0.0010,0.0005,0.0000,0.0000,0.0000,0.0000,0.0000,0.0010,...,0.0000,0.0010,0.0005,0.0010,0.0010,0.0005,0.0025,0.0000,0.0000,0.0000
8,0.0000,0.0000,0.0000,0.0005,0.0000,0.0005,0.0005,0.0000,0.0000,0.0005,...,0.0015,0.0000,0.0010,0.0000,0.0005,0.0000,0.0010,0.0000,0.0005,0.0000
9,0.0015,0.0000,0.0000,0.0005,0.0005,0.0000,0.0000,0.0005,0.0000,0.0000,...,0.0015,0.0015,0.0005,0.0005,0.0010,0.0005,0.0015,0.0000,0.0000,0.0010
